In [1]:
import pandas as pd
# fit and evaluate an AR model
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy
from math import sqrt

In [2]:
def difference(dataset):
    diff = []
    for row in dataset:
        diff_row = []
        for i in range(0, len(row)):
            value = int(row[i].replace(',', '')) #int(row[i-1].replace(',', ''))
            diff_row.append(value)
        diff.append(diff_row)
    return numpy.array(diff)

# Make a prediction give regression coefficients and lag obs
def predict(coef, history):
    yhat = coef[0]
    for i in range(1, len(coef)):
        yhat += coef[i] * history[-i]
    return yhat
 
df = read_csv('medicare_enrollment_state.csv', header=0, index_col=0)


In [3]:
df.dtypes

2010    object
2011    object
2012    object
2013    object
2014    object
2015    object
2016    object
2017    object
2018    object
dtype: object

In [4]:
df

,2010,2011,2012,2013,2014,2015,2016,2017,2018
Location,,,,,,,,,
Alabama,"833,737","852,740","881,686","910,826","934,681","957,230","982,978","1,003,155","1,045,702"
Alaska,"64,031","65,356","69,301","73,033","76,102","79,432","84,479","87,935","95,483"
Arizona,"903,761","933,435","977,447","1,021,663","1,063,224","1,106,623","1,155,167","1,199,968","1,279,721"
Arkansas,"525,417","536,817","552,375","567,391","579,516","590,707","604,934","615,859","646,094"
California,"4,669,733","4,806,469","5,000,198","5,204,013","5,378,580","5,558,531","5,755,043","5,915,868","6,224,832"
Colorado,"610,002","631,387","667,277","703,686","737,161","768,741","814,593","844,616","911,645"
Connecticut,"560,693","571,020","586,545","602,359","614,302","625,186","638,096","649,953","679,355"
Delaware,"146,006","151,077","157,289","164,349","170,630","176,884","183,704","190,055","201,656"
District of Columbia,"77,073","78,860","81,260","83,575","85,358","87,367","89,688","91,296","95,334"


In [5]:
# split dataset
X = difference(df.values)
size = int(len(X) * .66)
train, test = X[0:size], X[size:]

In [6]:
train

array([[ 833737,  852740,  881686,  910826,  934681,  957230,  982978,
        1003155, 1045702],
       [  64031,   65356,   69301,   73033,   76102,   79432,   84479,
          87935,   95483],
       [ 903761,  933435,  977447, 1021663, 1063224, 1106623, 1155167,
        1199968, 1279721],
       [ 525417,  536817,  552375,  567391,  579516,  590707,  604934,
         615859,  646094],
       [4669733, 4806469, 5000198, 5204013, 5378580, 5558531, 5755043,
        5915868, 6224832],
       [ 610002,  631387,  667277,  703686,  737161,  768741,  814593,
         844616,  911645],
       [ 560693,  571020,  586545,  602359,  614302,  625186,  638096,
         649953,  679355],
       [ 146006,  151077,  157289,  164349,  170630,  176884,  183704,
         190055,  201656],
       [  77073,   78860,   81260,   83575,   85358,   87367,   89688,
          91296,   95334],
       [3304763, 3390801, 3527830, 3663701, 3795696, 3931539, 4111667,
        4239320, 4515510],
       [1212748, 125

In [7]:
model = LinearRegression().fit(train[:, 0:4], train[:, 4:5])

In [8]:
r2 = model.score(test[:, 0:4], test[:, 4:5])
print(r2)

0.999996345836335


In [9]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [10]:
preds_2019 = model.predict(train[:, 1:5])
print(preds_2019)

[[955641.126521]
 [78808.273660]
 [1099987.050354]
 [588956.157184]
 [5543030.332335]
 [768637.304925]
 [624243.342918]
 [176268.876359]
 [86489.488251]
 [3904856.826801]
 [1495396.132719]
 [240221.306063]
 [274545.456610]
 [2048766.891036]
 [1133305.985822]
 [563145.788242]
 [480689.423371]
 [853650.448995]
 [777959.090315]
 [303014.390594]
 [918996.861409]
 [1200016.993921]
 [1876993.816419]
 [897457.417431]
 [552431.579503]
 [1123465.960767]
 [197371.133268]
 [306874.371545]
 [437967.944505]
 [260674.384603]
 [1476432.392649]
 [364214.133908]
 [3306355.801945]
 [1738749.822973]
 [115395.581789]
 [2123652.680521]]


In [11]:
df1 = pd.Series()
#df1['loc']=df['Location']
df1['pred19']=preds_2019
df1.head()
df.to_csv('2019.csv')

In [ ]:
#Do not run
# train autoregression
window = 6
model = ARIMA(train, order=(5, 2, 2))
model_fit = model.fit()
coef = model_fit.params
# walk forward over time steps in test
history = [train[i] for i in range(len(train))]
predictions = list()
for t in range(len(test)):
	yhat = predict(coef, history)
	obs = test[t]
	predictions.append(yhat)
	history.append(obs)
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
#Do not run
rmse = sqrt(mean_squared_error(test[:, 4:5], predictions))
print('Test RMSE: %.3f' % rmse)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()